# Situational Analysis: Teachers' Efficacy in Using Digital Resources for STEM Subjects in Kenyan Schools

## Date: 16/05/2025

## Analyst : Simiyu David Masai
---

# Phase 1: BUSINESS UNDERSTANDING

## 1.1 Project Overview

- This project analyzes the efficacy of teachers in Kenyan Junior and Secondary schools in utilizing digital resources for STEM (Science, Technology, Engineering, and Mathematics) education.
- The analysis is based on data collected through Head of Institution interviews, teacher questionnaires, learner Focus Group Discussions (FGDs), and lesson observations.
- The primary goal is to **understand the current landscape, identify challenges, and provide actionable insights for stakeholders like CEMASTEA (Centre for Mathematics, Science and Technology Education in Africa) and the Ministry of Education, Kenya**.

This research aims to address the following:
* **General Objective:**
- To assess the teachers’ efficacy on the use of digital resources for learning STEM subjects in Kenyan schools.
* **Specific Objectives:**
    1.  To investigate the relationship between teacher training (TPD) in STEM and their ability to effectively use digital resources.
    2.  To establish the influence of the use of digital resources on the teacher's instructional practices in STEM subjects.
    3.  To establish teachers’ awareness on the available digital resources for STEM subjects.
    4.  To assess the extent of support provided to teachers for adopting digital resources in teaching STEM subjects.

**Keywords:** STEM Education, Digital Resources, Teacher Efficacy, Kenya, Educational Technology, Teacher Training, Instructional Practices, CEMASTEA, CBC Kenya.

---

## 1.2 KEY STAKEHOLDERS

1. MINISTRY OF EDUCATION
2. CENTRE OF MATHEMATICS SCIENCE AND TECHNOLOGY EDUCATION IN AFRICA
3. SCHOOL ADMINSTRATORS,TEACHERS AND LEARNERS
4. RESEARCH COMMUNITIES



---
## 1.3 DATA SOURCES


In [19]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
#import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 2. DATA UNDERSTANDING

## DATA FROM JUNIOR SECONDARY


In [21]:
#load datasets
#paths to datasets
js1=pd.read_excel('/content/drive/MyDrive/Research on teachers Efficacy on Digital resources/JS Data/FGD-JS.xlsx')
js2=pd.read_excel('/content/drive/MyDrive/Research on teachers Efficacy on Digital resources/JS Data/HoI-JS.xlsx')
js3=pd.read_excel('/content/drive/MyDrive/Research on teachers Efficacy on Digital resources/JS Data/LO - JS .xlsx')
questions=pd.read_excel("/content/drive/MyDrive/Research on teachers Efficacy on Digital resources/JS Data/Teachers' Questionnaire-JS.xlsx")

In [22]:
#checking dataframes
questions.head()

,1. Have you been observed in class (for this research),2.a. Name of School,2.b. County,2.c. Sub-County,3. Gender,4.a. Type of school: Category of school,4.b. Type of school: Nature of school,4.c. Type of school:,5. Indicate your age range:,6. Teaching experience?,...,Statement [11. I encourage learners to ask questions on the concepts developed using the digital resources],Statement [12. I give learners enough time to interact with digital resources],Statement [13. I give learners assignments on concepts learned using the digital resources],Statement [14. I use digital resources to assess learning],15. Which digital resources do you commonly use in your lessons? \na) Hardware:,15. Which digital resources do you commonly use in your lessons? \nb) Software (Apps/videos/animation/simulations/virtual labs/Audio etc.):,16. How has the use of digital resources influenced my teaching practices?,17. What type of support do you receive from the school leadership in the use of digital resources?,18. (a) Briefly explain how teacher professional development has helped you in the use digital resources,18. (b) Suggest training needs to be addressed on the use of digital resources in future Teacher Professional Development program
0,No,ST.Thomas,Kilifi,Kilifi North,Female,Public,Regular,Junior School (JS),31-40 year,11 - 15 years,...,NaN,NaN,2.0,3.0,Tablets,"Videos, audio and apps",It has improved learner participation and unde...,Provision of enough digital devices,It has improved on my methodology skills.,Retooling
1,No,ST.Thomas,Kilifi,Kilifi North,Female,Public,Regular,Junior School (JS),21-30 years,NaN,...,5.0,4.0,4.0,4.0,NaN,"Videos, Animation, audios",It has made my teaching enjoyable and learners...,Provision of enough digital device,NaN,Retooling \nCEMASTEA Training
2,No,Kakamega Pri. Junior school,Kakamega,Kakamega Central,Male,NaN,NaN,Junior School (JS),21-30 years,5 years and below,...,4.0,5.0,5.0,2.0,"Tablets, smart phone, laptop, computer and pro...","Videos, virtual labs",It has made learner to enjoy my lessons especi...,"The school have provided us with tablet, a pro...","Through tea her professional development,I lea...",Increase time allocation for training of use o...
3,No,AIC KOSIRAI ACADEMY,Nandi,Chesumei,Male,Public,Regular,Junior School (JS),31-40 year,6 - 10 years,...,3.0,3.0,2.0,3.0,Mobile phone,You tube videos,Helps simplify some concepts,NaN,It has helped me incorporate digital literacy ...,NaN
4,No,ST.Thomas,Kilifi,Kilifi North,Female,Public,Regular,Junior School (JS),21-30 years,5 years and below,...,5.0,5.0,5.0,5.0,Laptops,"Videos, Apps, Audio",It has enhanced collaborative learning and enc...,Resources allocation \ncapacity building,It ha enhanced skills \nIt has made lesson pre...,Workshops


In [23]:
questions.tail()

,1. Have you been observed in class (for this research),2.a. Name of School,2.b. County,2.c. Sub-County,3. Gender,4.a. Type of school: Category of school,4.b. Type of school: Nature of school,4.c. Type of school:,5. Indicate your age range:,6. Teaching experience?,...,Statement [11. I encourage learners to ask questions on the concepts developed using the digital resources],Statement [12. I give learners enough time to interact with digital resources],Statement [13. I give learners assignments on concepts learned using the digital resources],Statement [14. I use digital resources to assess learning],15. Which digital resources do you commonly use in your lessons? \na) Hardware:,15. Which digital resources do you commonly use in your lessons? \nb) Software (Apps/videos/animation/simulations/virtual labs/Audio etc.):,16. How has the use of digital resources influenced my teaching practices?,17. What type of support do you receive from the school leadership in the use of digital resources?,18. (a) Briefly explain how teacher professional development has helped you in the use digital resources,18. (b) Suggest training needs to be addressed on the use of digital resources in future Teacher Professional Development program
461,Yes,Mandera DEB,Mandera,Mandera East,Male,Public,Regular,Junior School (JS),21-30 years,5 years and below,...,4.0,5.0,5.0,3.0,"Tablets, projector, laptops, smartphones","videos, audio, apps",I has helped me to deliver the lesson in a mor...,They give us digital devices,It lays the foundation for effective lesson pr...,RLM Training
462,No,Daua Junior,Mandera,Mandera East,Male,Public,Regular,Junior School (JS),31-40 year,6 - 10 years,...,5.0,4.0,4.0,5.0,Projector,video clips,It has improved content delivery and saving time,provision of digital devices,Helps in planning school/teacher activities; l...,Provision of adequate digital devices and plat...
463,No,Daua Junior School,Mandera,Mandera East,Female,Public,Regular,Junior School (JS),21-30 years,5 years and below,...,3.0,3.0,3.0,2.0,printed notes,audio,It has made it easier,Projector,Not indicated,Not indicated
464,No,Moi Girls,Mandera,Mandera East,Female,Public,Regular,Junior School (JS),21-30 years,5 years and below,...,5.0,5.0,5.0,2.0,Phone,apps and videos,By doing research on concepts,Provision of digital resources like computers,Not Indicated,ICT training
465,No,Daua Integrated,Mandera,Mandera East,Female,Public,Regular & SNE,Junior School (JS),Above 50 years,Above 20 years,...,5.0,4.0,4.0,4.0,smartphone,videos and animations,has motivated and captured learners' interest,Moral support and also availability of the dig...,Equipped me with the necessary skills in the u...,More trainings on ICT integration in schools; ...


In [24]:
#checking shape
shape=questions.shape
print(f'Dataframe has {shape[0]} rows and {shape[1]} columns')

Dataframe has 466 rows and 35 columns


In [26]:
#checking datatypes
questions.dtypes

,0
1. Have you been observed in class (for this research),object
2.a. Name of School,object
2.b. County,object
2.c. Sub-County,object
3. Gender,object
4.a. Type of school: Category of school,object
4.b. Type of school: Nature of school,object
4.c. Type of school:,object
5. Indicate your age range:,object
6. Teaching experience?,object


In [25]:
#checking the columns
questions.columns

Index(['1. Have you been observed in class (for this  research) ',
       '2.a. Name of School ', '2.b. County', '2.c. Sub-County', '3. Gender',
       '4.a. Type of school: Category of school ',
       '4.b. Type of school: Nature of school ', '4.c. Type of school:',
       '5. Indicate your age range: ', '6. Teaching experience? ',
       '7. Highest level of academic qualification: ',
       '8. How many lessons do you teach per week? ', '9. Employer:',
       '10. Have you undergone any teacher professional development training? ',
       'If YES, for how many times? ',
       'Statement  [1. I source for digital resources to use during my lesson]',
       'Statement  [2. I collaborate with colleagues in sourcing for digital resources]',
       'Statement  [3. I develop digital resources for use during the lessons]',
       'Statement  [4. I try-out the digital resources before the lessons]',
       'Statement  [5. The school provides adequate digital tools for use in the lessons]'

In [27]:
#CHECKING NULL VALUES
questions.isnull().sum()

,0
1. Have you been observed in class (for this research),26
2.a. Name of School,0
2.b. County,1
2.c. Sub-County,4
3. Gender,2
4.a. Type of school: Category of school,5
4.b. Type of school: Nature of school,13
4.c. Type of school:,0
5. Indicate your age range:,0
6. Teaching experience?,7


### filling null values
- fill this columns with mean ['1. Have you been observed in class (for this research)','6. Teaching experience?','8. How many lessons do you teach per week?','If YES, for how many times?']
- fill with mode ['1. Have you been observed in class (for this research)','7. Highest level of academic qualification:','9. Employer:','10. Have you undergone any teacher professional development training?','Statement [1. I source for digital resources to use during my lesson]','Statement [2. I collaborate with colleagues in sourcing for digital resources]','Statement [3. I develop digital resources for use during the lessons]']
- columns for research ['4.a. Type of school: Category of school	5', '4.b. Type of school: Nature of school']

In [28]:
mode_value = questions['1. Have you been observed in class (for this  research) '].mode()[0]
questions['1. Have you been observed in class (for this  research) '] = questions['1. Have you been observed in class (for this  research) '].fillna(mode_value)

In [29]:
#checking
questions['1. Have you been observed in class (for this  research) '].isna().sum()

np.int64(0)

In [33]:
#using name of school to fill its geographical location
geog= questions[['2.a. Name of School ', '2.b. County', '2.c. Sub-County']]
geog.head()

,2.a. Name of School,2.b. County,2.c. Sub-County
0,ST.Thomas,Kilifi,Kilifi North
1,ST.Thomas,Kilifi,Kilifi North
2,Kakamega Pri. Junior school,Kakamega,Kakamega Central
3,AIC KOSIRAI ACADEMY,Nandi,Chesumei
4,ST.Thomas,Kilifi,Kilifi North


In [36]:
# Show all rows with at least one null value
missing=geog[geog.isnull().any(axis=1)]
#unique schools with missing geographical location
missing['2.a. Name of School '].unique()

array(['Sheikh Khalifa jss ', 'Kapsabet Girls ', "St. Patrick's Val",
       'Mikindani primary ', 'Munanda '], dtype=object)

In [37]:
#filling null values in school geographical columns
# Mapping of schools to their correct county and sub-county
schools = ['Sheikh Khalifa jss ', 'Kapsabet Girls ', "St. Patrick's Val", 'Mikindani primary ', 'Munanda ']
counties = ['Mombasa', 'Nandi', 'Homa Bay', 'Mombasa', 'Nakuru']
sub_counties = ['Nyali', 'Nandi Central', 'Rangwe', 'Changamwe', 'Gilgil']

county_map = dict(zip(schools, counties))
subcounty_map = dict(zip(schools, sub_counties))

# Fill missing values for the relevant schools
for school in schools:
    mask = questions['2.a. Name of School '] == school

    # Only fill where values are currently null
    questions.loc[mask & questions['2.b. County'].isnull(), '2.b. County'] = county_map[school]
    questions.loc[mask & questions['2.c. Sub-County'].isnull(), '2.c. Sub-County'] = subcounty_map[school]

In [40]:
#checking
questions[['2.a. Name of School ', '2.b. County', '2.c. Sub-County']].isna().sum()

,0
2.a. Name of School,0
2.b. County,0
2.c. Sub-County,0


- dealth with all null values

In [42]:
nature=questions[['2.a. Name of School ', '3. Gender',
       '4.a. Type of school: Category of school ',
       '4.b. Type of school: Nature of school ']]
nature.head()

,2.a. Name of School,3. Gender,4.a. Type of school: Category of school,4.b. Type of school: Nature of school
0,ST.Thomas,Female,Public,Regular
1,ST.Thomas,Female,Public,Regular
2,Kakamega Pri. Junior school,Male,NaN,NaN
3,AIC KOSIRAI ACADEMY,Male,Public,Regular
4,ST.Thomas,Female,Public,Regular


In [48]:
nature.isna().sum()

,0
2.a. Name of School,0
3. Gender,2
4.a. Type of school: Category of school,5
4.b. Type of school: Nature of school,13


In [52]:
questions['4.b. Type of school: Nature of school '] = questions['4.b. Type of school: Nature of school '].fillna('Regular')
questions['4.a. Type of school: Category of school '] = questions['4.a. Type of school: Category of school '].fillna('Public')
questions['3. Gender'] = questions['3. Gender'].fillna('Male')

In [54]:
#checking
nature=questions[['2.a. Name of School ', '3. Gender',
       '4.a. Type of school: Category of school ',
       '4.b. Type of school: Nature of school ']]
nature.isna().sum()

,0
2.a. Name of School,0
3. Gender,0
4.a. Type of school: Category of school,0
4.b. Type of school: Nature of school,0


- Dealt with all null values

In [62]:
questions['6. Teaching experience? '] = questions['6. Teaching experience? '].fillna('5 years and below  ')
questions['7. Highest level of academic qualification: '] = questions['7. Highest level of academic qualification: '].fillna('Degree  ')
questions['8. How many lessons do you teach per week? '] = questions['8. How many lessons do you teach per week? '].fillna(28)
questions['9. Employer:'] = questions['9. Employer:'].fillna('TSC')
questions['10. Have you undergone any teacher professional development training? '] = questions['10. Have you undergone any teacher professional development training? '].fillna('Yes')
questions['If YES, for how many times? '] = questions['If YES, for how many times? '].fillna('More than three times')

In [65]:
questions['If YES, for how many times? '] = questions['If YES, for how many times? '].fillna('More than three times')


In [67]:
columns = [
    'Statement  [1. I source for digital resources to use during my lesson]',
    'Statement  [2. I collaborate with colleagues in sourcing for digital resources]',
    'Statement  [3. I develop digital resources for use during the lessons]',
    'Statement  [4. I try-out the digital resources before the lessons]',
    'Statement  [5. The school provides adequate digital tools for use in the lessons]',
    'Statement  [6. The school administration supports me to use digital resources for learning]',
    'Statement  [7. The professional development opportunities provided enhance my knowledge and skills in using digital resources]',
    'Statement [8. I give all learners equal opportunities in the use of the digital resources]',
    'Statement [9. I ask learners questions to guide learners’ interaction with the digital resources]',
    'Statement [10. I engage my learners in group discussions to share their ideas with their peers as they interact with the digital resources]',
    'Statement [11. I encourage learners to ask questions on the concepts developed using the digital resources]',
    'Statement [12. I give learners enough time to interact with digital resources]',
    'Statement [13. I give learners assignments on concepts learned using  the digital resources]',
    'Statement [14. I use digital resources   to assess learning]',
    '15. Which digital resources do you commonly use in your lessons? \na) Hardware:',
    '15. Which digital resources do you commonly use in your lessons? \nb)  Software (Apps/videos/animation/simulations/virtual labs/Audio etc.):',
    '16.  How has the use of digital resources influenced my teaching practices?  ',
    '17. What type of support do you receive from the school leadership in the use of digital resources? ',
    '18. (a)  Briefly explain how teacher  professional development has helped you in the use digital resources ',
    '18. (b)  Suggest training needs to be addressed on the use of digital resources in future Teacher Professional Development program '
]

for col in columns:
    if questions[col].dtype in ['float64', 'int64']:
        mean_val = questions[col].mean()
        questions[col] = questions[col].fillna(mean_val)
    elif questions[col].dtype == 'O':  # object (string/categorical)
        mode_val = questions[col].mode()[0]  # mode() returns a Series
        questions[col] = questions[col].fillna(mode_val)
    else:
        # If the dtype is something else, you can add more logic or skip
        pass

# Optional: Check that missing values have been filled
questions[columns].isnull().sum()

,0
Statement [1. I source for digital resources to use during my lesson],0
Statement [2. I collaborate with colleagues in sourcing for digital resources],0
Statement [3. I develop digital resources for use during the lessons],0
Statement [4. I try-out the digital resources before the lessons],0
Statement [5. The school provides adequate digital tools for use in the lessons],0
Statement [6. The school administration supports me to use digital resources for learning],0
Statement [7. The professional development opportunities provided enhance my knowledge and skills in using digital resources],0
Statement [8. I give all learners equal opportunities in the use of the digital resources],0
Statement [9. I ask learners questions to guide learners’ interaction with the digital resources],0
Statement [10. I engage my learners in group discussions to share their ideas with their peers as they interact with the digital resources],0


- Dealt with all null values

In [68]:
#saving cleaned dataframe as questionnaire_cleaned
questionnaire_cleaned = questions.copy()
#saving it as an excel file
questionnaire_cleaned.to_excel('questionnaire_cleaned.xlsx', index=False)

## SECOND DATAFRAME

In [69]:
js1.head()

,1.a. Name of School,1.b. County,1.c. Sub-County,2. Level of the school:,3. Category:,4.a. Type of school:,4.b. Type of school:,5. Nature of School:,"1. Name some of the digital resources your teachers use to teach integrated science, Mathematics, Pre-technical, and Agriculture, Physics, Biology or Chemistry lessons.","2. Tell me some specific lesson(s) you were taught using digital resources. (Probe the specific content area in Integrated science, Mathematics, Pre-technical, and Agriculture/ Physics, Biology or Chemistry)","3. How often are digital resources used during your Science or mathematics lessons?(To probe how many lessons, digital resources have been used, in this term- Science, mathematics, Pre-technical, and Agriculture, Physics, Biology or Chemistry lessons)",4. Does your teacher allow you to interact with digital resources? (Probe on the mode of interaction with digital resources),"5.a. How do you feel when your teacher uses digital resources in the lesson? (Probe for interest, or easy understanding)","5.b. Can you share an example of a digital resource that made the lesson exciting, enjoyable, or easier to understand? (Probe for Integrated science, mathematics, pre-technical, and Agriculture, Physics, Biology or Chemistry)","6. Does your teacher give you assignments on the main idea learnt when using digital resources in a lesson? (If YES, explain how the assignment help you to recall the learnt idea)"
0,Ngererit Primary and Junior School,Bomet,Konoin,Junior School,Public,Day,Mixed Boys/Girls,Regular,Phones\nLaptops\nComputers\nScientific Calcula...,Laptop - Teaching Plants in Integrated Science...,About 2 to 3 lessons per week in lessons of In...,Rarely. Learners shown the outcome on the phones,"Feeling happy, better understanding of the con...",Easier to understanding by seeing photos of or...,Rarely
1,St. Mary Mixed Day and Boarding Primary & Juni...,Bomet,Chepalungu,Junior School,Private,Mixed Day/Boarding,Mixed Boys/Girls,Regular,a) Laptops\nb) Computers\nc) Mobile phones\nd)...,"Very often, at least once a week\na) Organic F...",Yes. At least once a week,Yes. To search for content in the internet,"Excited, happy, learning interesting hence mor...","a) Types of Computers, Parts of a Computer - P...",Yes. Learners do further research
2,Kiplabotwa Primary & Junior School,Bomet,Bomet East,Junior School,Public,Day,Mixed Boys/Girls,Regular & SNE,a) Tablets\nb) Laptops\nc) Note Books\nd) Desk...,"a) Structure of an Atom, Living Things - Integ...",Twice per week majorly in Mathematics and Scie...,Yes. Through google searches and YouTube,Lessons interesting. More and better understan...,a) Living Things - Integrated Science\nb) Stru...,Yes.
3,Sotik JS,Bomet,Sotik,Junior School,Public,Day,Mixed Boys/Girls,Regular,"Computers, tablets, markbook, smartphones, lap...","Lab safety in integrated science , how to crea...",Regularly,Yes. We are given tablets to search for digita...,"Happy, enjoyable, interesting and makes it eas...",Tablets and use of projector to project video,Given a link to copy and use it later to acces...
4,Chelsa Comprehensive JS Academy,Bomet,Bomet central,Junior School,Private,Mixed Day/Boarding,Mixed Boys/Girls,Regular,"Desktops, laptops, projectors, tablets, markbo...",Scratch and composite materials in pre-technic...,Average of 4 times per week. But for mathemati...,Interact with computer in computer lab to disc...,"Motivated, lesson is lively, understand more,f...","Computers, smartphones, laptops, projectors",Yes. Assignment to search for more information...


In [70]:
js1.tail()

,1.a. Name of School,1.b. County,1.c. Sub-County,2. Level of the school:,3. Category:,4.a. Type of school:,4.b. Type of school:,5. Nature of School:,"1. Name some of the digital resources your teachers use to teach integrated science, Mathematics, Pre-technical, and Agriculture, Physics, Biology or Chemistry lessons.","2. Tell me some specific lesson(s) you were taught using digital resources. (Probe the specific content area in Integrated science, Mathematics, Pre-technical, and Agriculture/ Physics, Biology or Chemistry)","3. How often are digital resources used during your Science or mathematics lessons?(To probe how many lessons, digital resources have been used, in this term- Science, mathematics, Pre-technical, and Agriculture, Physics, Biology or Chemistry lessons)",4. Does your teacher allow you to interact with digital resources? (Probe on the mode of interaction with digital resources),"5.a. How do you feel when your teacher uses digital resources in the lesson? (Probe for interest, or easy understanding)","5.b. Can you share an example of a digital resource that made the lesson exciting, enjoyable, or easier to understand? (Probe for Integrated science, mathematics, pre-technical, and Agriculture, Physics, Biology or Chemistry)","6. Does your teacher give you assignments on the main idea learnt when using digital resources in a lesson? (If YES, explain how the assignment help you to recall the learnt idea)"
95,Bar Union,Kisumu,Kisumu West,Junior School,Public,Day,Mixed Boys/Girls,Regular & SNE,"Tablets, laptops, projectors, mobile phones","Pictures, on internal structure of a leaf, tab...",Thrice a week in science and every day in maths,Yes. Gives us the digital devices even us we c...,"Good, get experience, more knowledge","Agriculture , integrated farming, drew images ...","Yes, in digital corner,helps get more practice..."
96,Nyagweno JS,Kisumu,Nyakach,Junior School,Public,Day,Mixed Boys/Girls,Regular,"Calculators, tablets, laptops, desktops, proje...","Agriculture, methods of taking care of square ...",Twice a week for all,Yes,"Great, we understand better",Integrated Science: introduction to plants and...,Yes. Very useful in helping us understand better
97,Talon Academy,West Pokot,Kipkomo,Junior School,Private,Mixed Day/Boarding,Mixed Boys/Girls,Regular,Ipad; laptops; tablets; smart phones; calculat...,Mathematics - squares and square roots\nAgricu...,Integrated Science and Pre-technical studies- ...,Yes. Operating the digital resources,Happy- It enables me to know how to use the ta...,"Tablets, ipads as we search for more informati...",Yes. Search for meaning of new words like cata...
98,Osodo Primary,Homa Bay,Rachuonyo North,Junior School,Public,Day,Mixed Boys/Girls,Regular & SNE,Computer,Pre-technical: Components of the components,Rarely: Pretech- Internet,"Yes, sometimes",Interesting,No answer,Researching
99,Fikow Junior School,Mandera,Khalalio,Junior School,Public,Day,Mixed Boys/Girls,Regular,"Tablets, laptop, mobile phones, projector",Integrated Science -Part of a computer; Englis...,three times in a term.,"Yes, we were allowed to use it in groups.",We feel good because it makes us understand th...,"Tablets, mobile phones, projector, laptop",no


In [71]:
#checking the shape
shape=js1.shape
print(f'Dataframe has {shape[0]} rows and {shape[1]} columns')

Dataframe has 100 rows and 15 columns


In [72]:
#checking columns
js1.columns

Index(['1.a. Name of School', '1.b. County', '1.c. Sub-County',
       '2.  Level of the school:', '3. Category:', '4.a. Type of school: ',
       '4.b.  Type of school:', '5. Nature of School:',
       '1.  Name some of the digital resources your teachers use to teach integrated science, Mathematics, Pre-technical, and Agriculture, Physics, Biology or Chemistry lessons.',
       '2. Tell me some specific lesson(s) you were taught using digital resources. (Probe  the specific content area in Integrated science, Mathematics, Pre-technical, and Agriculture/ Physics, Biology or Chemistry) ',
       '3. How often are digital resources used during your Science or mathematics lessons?(To probe how many lessons, digital resources have been used, in this term-  Science, mathematics, Pre-technical, and Agriculture, Physics, Biology or Chemistry lessons) ',
       '4. Does your teacher allow you to interact with digital resources? (Probe on the mode of interaction with digital resources)  ',
   

In [73]:
#checking null values
js1.isna().sum()

,0
1.a. Name of School,0
1.b. County,0
1.c. Sub-County,0
2. Level of the school:,0
3. Category:,0
4.a. Type of school:,0
4.b. Type of school:,0
5. Nature of School:,0
"1. Name some of the digital resources your teachers use to teach integrated science, Mathematics, Pre-technical, and Agriculture, Physics, Biology or Chemistry lessons.",0
"2. Tell me some specific lesson(s) you were taught using digital resources. (Probe the specific content area in Integrated science, Mathematics, Pre-technical, and Agriculture/ Physics, Biology or Chemistry)",0


- Only one column has a null value
NEXT STEPS:
- Checking nature of null values

In [75]:
#column with null value
missing=js1[ '4. Does your teacher allow you to interact with digital resources? (Probe on the mode of interaction with digital resources)  ']

In [76]:
#unique values in column
missing.unique()

array(['Rarely. Learners shown the outcome on the phones',
       'Yes. To search for content in the internet',
       'Yes. Through google searches and YouTube',
       'Yes. We are given tablets to search for digital content , ',
       'Interact with computer in computer lab to discover,discuss questions in groups using computers ',
       'In pretechincal studies learners are allowed to manupilate the devices, Agriculture and maths learners only observe,',
       'physics-once a week, biology and chemistry -connect the equipment and learners observe and manipulate, physics-only observation, In pre-techincal studies learners manipulate computers',
       'Learners are allowed to manipulate the digital devices',
       'learners interact with the resources',
       'Yes- learners operate the devices with the guidance of the teacher',
       'Yes. The teacher guides the learners to interact with the digital resources. ',
       'No', 'Yes. The teacher allow us to use tablets.',
      

- open-ended question
- drop row with null value

In [77]:
#drop row with null value in dataframe
js1.dropna(inplace=True)

In [78]:
#checking
js1.isna().sum()

,0
1.a. Name of School,0
1.b. County,0
1.c. Sub-County,0
2. Level of the school:,0
3. Category:,0
4.a. Type of school:,0
4.b. Type of school:,0
5. Nature of School:,0
"1. Name some of the digital resources your teachers use to teach integrated science, Mathematics, Pre-technical, and Agriculture, Physics, Biology or Chemistry lessons.",0
"2. Tell me some specific lesson(s) you were taught using digital resources. (Probe the specific content area in Integrated science, Mathematics, Pre-technical, and Agriculture/ Physics, Biology or Chemistry)",0


In [79]:
#saving cleaned dataframe as questionnaire_cleaned
FGD_JS_cleaned = js1.copy()
#saving it as an excel file
FGD_JS_cleaned.to_excel('FGD_JS_cleaned.xlsx', index=False)


### NEXT DATFRAME

In [80]:
js2.head()

,1.a. Name of the school:,JS/Sec,1.b. Name of your County,1.c. Sub-County,2. Head of Institution’s gender:,3.a. Type of school:,3.b. Type of school:,4.a. Category of School:,4.b. Nature of School:,5. For how long have you been a Head of Institution?,...,5.a. Printers (Quantity),5.b. Printers,6.a. Digital cameras (Quantity),6.b. Digital cameras,7.a. School mobile phone(s) (Quantity),7.b. School mobile phone(s),8.a. Software applications (phet) (Quantity),8.b. Software applications,9. Others:,10. Internet connectivity
0,Ngarerit JS,JS,Bomet,Konoin,Male,Day,Mixed Boys/Girls,Public,Regular,6-10 years,...,NaN,Not Adequate,NaN,Not adequate,NaN,Not Adequate,NaN,Not Adequate,NaN,Not Available
1,Sotik Primary & Junior School,JS,Bomet,Sotik,Male,Day,Mixed Boys/Girls,Public,Regular,6-10 years,...,2,Not Adequate,0,Not adequate,0,Not Adequate,0,Not Adequate,NaN,Available
2,Kiplabotwa Primary & Junior School,JS,Bomet,Bomet East,Male,Day,Mixed Boys/Girls,Public,Regular & SNE,6-10 years,...,1,Not Adequate,0,Not adequate,0,Not Adequate,0,Not Adequate,NaN,Available
3,St. Mary's Mixed Day and Boarding Primary & Ju...,JS,Bomet,Chepalungu,Female,Mixed Day/Boarding,Mixed Boys/Girls,Private,Regular,6-10 years,...,5,Not Adequate,1,Not adequate,0,Not Adequate,0,Not Adequate,NaN,Available
4,Chelsa Academy - Primary & Junior School,JS,Bomet,Bomet Central,Male,Mixed Day/Boarding,Mixed Boys/Girls,Private,Regular,1-5 years,...,2,Adequate,0,Not adequate,0,Not Adequate,0,Not Adequate,NaN,Available


In [81]:
js2.tail()

,1.a. Name of the school:,JS/Sec,1.b. Name of your County,1.c. Sub-County,2. Head of Institution’s gender:,3.a. Type of school:,3.b. Type of school:,4.a. Category of School:,4.b. Nature of School:,5. For how long have you been a Head of Institution?,...,5.a. Printers (Quantity),5.b. Printers,6.a. Digital cameras (Quantity),6.b. Digital cameras,7.a. School mobile phone(s) (Quantity),7.b. School mobile phone(s),8.a. Software applications (phet) (Quantity),8.b. Software applications,9. Others:,10. Internet connectivity
94,Daua Integrated,JS,Mandera,Mandera East,Male,Mixed Day/Boarding,Mixed Boys/Girls,Public,Regular & SNE,16-20 years,...,4,Adequate,0,NaN,10,Adequate,0,NaN,none,Available
95,Mandera Junior Integrated Academy,JS,Mandera,Mandera East,Male,Day,Mixed Boys/Girls,Private,Regular,6-10 years,...,2,Adequate,0,NaN,0,NaN,0,NaN,"1 radio, adequate",Not Available
96,Kabichbich JS,JS,West Pokot,Pokot South,Male,Mixed Day/Boarding,Mixed Boys/Girls,Public,Regular,1-5 years,...,1,Not Adequate,0,Not adequate,0,Not Adequate,0,Not Adequate,0,Not Available
97,Fikow Primary and JS,JS,Mandera,Khalalio,Male,Day,Mixed Boys/Girls,Public,Regular,11-15 years,...,0,NaN,0,NaN,0,NaN,0,NaN,NaN,Not Available
98,NYAGWENO JS,JS,Kisumu,NYAKACH,Male,Day,Mixed Boys/Girls,Public,Regular,11-15 years,...,1,Not Adequate,0,Not adequate,0,Not Adequate,0,Not Adequate,NaN,Not Available


In [82]:
#checking shape
shape=js2.shape
print(f'Dataframe has {shape[0]} rows and {shape[1]} columns')

Dataframe has 99 rows and 41 columns


In [92]:
js2['3.b. Type of school:']

,3.b. Type of school:
0,Mixed Boys/Girls
1,Mixed Boys/Girls
2,Mixed Boys/Girls
3,Mixed Boys/Girls
4,Mixed Boys/Girls
...,...
94,Mixed Boys/Girls
95,Mixed Boys/Girls
96,Mixed Boys/Girls
97,Mixed Boys/Girls


In [84]:
#checking columns
js2.columns


Index(['1.a. Name of the school: ', 'JS/Sec', '1.b. Name of your County',
       '1.c. Sub-County', '2. Head of Institution’s gender:',
       '3.a. Type of school:', '3.b. Type of school:',
       '4.a. Category of School:', '4.b. Nature of School:',
       '5. For how long have you been a Head of Institution?',
       '6.a. Do your STEM teachers participate in professional development  training related to their subjects?',
       '6.b. Probe on how often  this happens and the kind of training',
       '7.a. Do you feel the training opportunities adequately equip teachers with the skills to use digital Resources in STEM effectively?',
       '7.b. Probe for why or why not',
       '8. What measures do you have in place to ensure teachers are updated with the current digital resources for STEM?  ( Probe about AI, virtual labs among others)',
       '9.a. Do your STEM teachers incorporate digital resources in their lessons?',
       '9.b. (Probe on frequency and cite instances) (In your

In [136]:
#checking null values
js2.isna().sum()

,0
1.a. Name of the school:,0
JS/Sec,0
1.b. Name of your County,0
1.c. Sub-County,0
2. Head of Institution’s gender:,0
3.a. Type of school:,0
3.b. Type of school:,0
4.a. Category of School:,0
4.b. Nature of School:,0
5. For how long have you been a Head of Institution?,0


In [137]:
#drop rows with null values
js2.dropna(inplace=True)

In [138]:
#checking
js2.isna().sum()

,0
1.a. Name of the school:,0
JS/Sec,0
1.b. Name of your County,0
1.c. Sub-County,0
2. Head of Institution’s gender:,0
3.a. Type of school:,0
3.b. Type of school:,0
4.a. Category of School:,0
4.b. Nature of School:,0
5. For how long have you been a Head of Institution?,0


In [139]:
#saving cleaned dataframe
HoI_JS_cleaned = js2.copy()
#saving it as an excel file
HoI_JS_cleaned.to_excel('HoI_JS_cleaned.xlsx', index=False)

## NEXT DATAFRAME

In [140]:
js3.head()

,1.a. School,1.b. County,1.c. Sub-County,2.a. Date,2.b. Time : From,2.c. Time : To,3.a. Name of the teacher,3.b. TSC No:,4. Gender,5. Professional qualifications:,...,"Innovative Use of Digital Resource [5. Digital resources were well-aligned with learning outcomes and learner’s needs (e.g., age-appropriate content, differentiated instruction).]",Assessment [6. The teacher provided timely and constructive feedback based on the learners interaction with the digital resource],Assessment [7. Learners had the opportunity to give feedback on the concept learnt based on the their interaction with the digital resource],Classroom Management [8. Teacher managed transitions in activities involving digital resources seamlessly],"Classroom Management [9. Learners remained focused on the task, as the teacher guides or redirects their behavior]","Digital Fluency [10. Teacher demonstrated proficiency in using digital resources to achieve lesson learning outcomes/objectives (e.g., confident navigation of software).]","Digital Fluency [11. Teacher resolved technical issues effectively during the lesson (e.g., troubleshooting connectivity problems-If none rate 3)]",1. How prepared was the teacher in terms of accessing and utilizing digital resources?,"2. Were there any notable challenges in using digital resources? If yes, describe them.",3. Suggest area(s) in which the teacher should be supported to enhance his/her skills in the use of digital resources in the learning process
0,Ngererit Pri and JS,Bomet,Konoin,2025-10-02 00:00:00,11:12:00,12:21:00,Mercy Chepkoech,1047101,Female,B Ed,...,NaN,1.0,1.0,1.0,2.0,2.0,3.0,less prepared,-yes. - one mobile phone against all learners ...,-exposure on softwares to use for teaching. - ...
1,Victory academy,Busia,Teso south,2025-02-10 00:00:00,14:00:00,14:40:00,Collins Ojuma,1083155,Male,B Ed,...,2.0,3.0,3.0,3.0,3.0,3.0,3.0,The teacher was well prepared to use computers...,Half the class could not access internet due t...,Teacher should be supported on use transformat...
2,nyamwanga,Busia,Butula,NaN,14:00:00,14:40:00,Wanga Steve,1010790,Male,B Ed,...,3.0,3.0,3.0,3.0,2.0,3.0,3.0,"prepared with laptop, projector video clip","uses were shown but there was no demonstratio,...",involve learners in manipulating resources edi...
3,Nyamwanga js,Busia,Butula,2024-02-12 00:00:00,14:00:00,14:40:00,Madam ndubi,875397,Female,Diploma in Ed,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,the tacher was not to use digital resources in...,NA,the should purpose to use digital resources in...
4,Busulere js,Busia,samia,2025-02-12 00:00:00,08:40:00,09:20:00,Susan Ogomaa,928554,Female,B Ed,...,3.0,3.0,3.0,3.0,3.0,2.0,3.0,had the digital resources set ready for the le...,surface for projection of the digital resource...,enhance the pedagogical skills of the teacher


In [141]:
js3.tail()

,1.a. School,1.b. County,1.c. Sub-County,2.a. Date,2.b. Time : From,2.c. Time : To,3.a. Name of the teacher,3.b. TSC No:,4. Gender,5. Professional qualifications:,...,"Innovative Use of Digital Resource [5. Digital resources were well-aligned with learning outcomes and learner’s needs (e.g., age-appropriate content, differentiated instruction).]",Assessment [6. The teacher provided timely and constructive feedback based on the learners interaction with the digital resource],Assessment [7. Learners had the opportunity to give feedback on the concept learnt based on the their interaction with the digital resource],Classroom Management [8. Teacher managed transitions in activities involving digital resources seamlessly],"Classroom Management [9. Learners remained focused on the task, as the teacher guides or redirects their behavior]","Digital Fluency [10. Teacher demonstrated proficiency in using digital resources to achieve lesson learning outcomes/objectives (e.g., confident navigation of software).]","Digital Fluency [11. Teacher resolved technical issues effectively during the lesson (e.g., troubleshooting connectivity problems-If none rate 3)]",1. How prepared was the teacher in terms of accessing and utilizing digital resources?,"2. Were there any notable challenges in using digital resources? If yes, describe them.",3. Suggest area(s) in which the teacher should be supported to enhance his/her skills in the use of digital resources in the learning process
96,MAKANDE,Mombasa,MVITA,2025-02-10 00:00:00,12:00:00,12:40:00,MOSES OBUYA,868937,Male,B Ed,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,Teacher did not use digital resources,none,try and use the digital resources to teach abs...
97,SHEIKH KHALIFA BIN ZAYED,Mombasa,NYALI,2025-02-10 00:00:00,14:20:00,15:00:00,NGAO OMAR,546380,Male,B Ed,...,3.0,2.0,2.0,3.0,3.0,3.0,3.0,The use of laptop and projector to present the...,none,The lesson was an assessment session.
98,MIKINDANI JUNIOR SCH,Mombasa,JOMVU,2025-02-12 00:00:00,11:40:00,12:20:00,JACINTA ATIENO,927217,Male,B Ed,...,2.0,1.0,1.0,1.0,2.0,1.0,1.0,"There were digital resources available, laptop...",the connection failed at some point,"prepapre in advance, use ther available resour..."
99,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [142]:
#shape
shape=js3.shape
print(f'Dataframe has {shape[0]} rows and {shape[1]} columns')

Dataframe has 101 rows and 43 columns


In [143]:
#columns
js3.columns

Index(['1.a. School', '1.b. County', '1.c. Sub-County', '2.a. Date',
       '2.b. Time : From', '2.c. Time : To', '3.a. Name of the teacher ',
       '3.b. TSC No:  ', '4. Gender', '5. Professional qualifications:  ',
       '6.1. Teaching subjects trained in (College subjects): ',
       '6.2. Teaching subjects trained in (College subjects):  ',
       '7. Employer ', '8. Have you attended any teacher training in STEM? ',
       '9. If yes, how many times have you undergone the training? ',
       '10. Grade/Form', 'JS/SS', '11. Subject ', '12. Strand/Topic ',
       '13. Sub-strand/Sub-Topic ', '14. Number of Learners: Total ',
       '15. Lesson Learning outcomes ', 'Learning Resources ',
       '16. Key Inquiry Question (KIQ) ',
       'Statements  [1. The teacher had a scheme of work]',
       'Statements  [2. Did the column of resources in the Schemes of work indicate digital resources]',
       'Statements  [3. The teacher had a lesson plan prepared as per the scheme of work]',


In [150]:
#drop null values in this columns['1.a. School', '1.b. County', '1.c. Sub-County']
js3.dropna(subset=['1.a. School', '1.b. County', '1.c. Sub-County'], inplace=True)

In [161]:
# #unique values in column
js3['16. Key Inquiry Question (KIQ) '].value_counts()

,count
16. Key Inquiry Question (KIQ),
none,26
What are the external parts of a leaf.,2
what are some of the apparatus and instruments used in lab,1
"What is book keeping, what is the importance of book keeping.",1
"what are the properties of a hexagon, what is the formula of calculating the area of a hexagon",1
...,...
What is an atom? What are some of the atomic particles in the atom?,1
How do we use fractions in real life?,1
What are the classifications of wood?,1


In [160]:
#filling columns
js3[ '16. Key Inquiry Question (KIQ) '] = js3[ '16. Key Inquiry Question (KIQ) '].fillna('none')

In [162]:
#checking null values
js3.isna().sum()

,0
1.a. School,0
1.b. County,0
1.c. Sub-County,0
2.a. Date,3
2.b. Time : From,2
2.c. Time : To,7
3.a. Name of the teacher,1
3.b. TSC No:,7
4. Gender,0
5. Professional qualifications:,0


In [163]:
#shape
shape=js3.shape
print(f'Dataframe has {shape[0]} rows and {shape[1]} columns')

Dataframe has 99 rows and 43 columns


In [164]:
#dropping null values
js3.dropna(inplace=True)
shape=js3.shape
print(f'Dataframe has {shape[0]} rows and {shape[1]} columns')

Dataframe has 49 rows and 43 columns


In [165]:
#saving
LO_JS_cleaned = js3.copy()
#saving it as an excel file
LO_JS_cleaned.to_excel('LO_JS_cleaned.xlsx', index=False)

- Through with JSS now onto Senior School Dataframes

# SENIOR SCHOOL DATAFRAME

In [167]:
# loading datasets
ss1=pd.read_excel('/content/drive/MyDrive/Research on teachers Efficacy on Digital resources/SS Data/FGD-Sec.xlsx')
ss2=pd.read_excel('/content/drive/MyDrive/Research on teachers Efficacy on Digital resources/SS Data/HoI - Sec.xlsx')
ss3=pd.read_excel('/content/drive/MyDrive/Research on teachers Efficacy on Digital resources/SS Data/LO - Sec.xlsx')
ss_questions=pd.read_excel("/content/drive/MyDrive/Research on teachers Efficacy on Digital resources/SS Data/Teacher Questionnaire-Sec Schl.xlsx")

- First Dataframe

In [168]:
ss1.head()

,1.a. Name of School,1.b. County,1.c. Sub-County,2. Level of the school:,3. Category:,4.a. Type of school:,4.b. Type of school:,5. Nature of School:,"1. Name some of the digital resources your teachers use to teach integrated science, Mathematics, Pre-technical, and Agriculture, Physics, Biology or Chemistry lessons.","2. Tell me some specific lesson(s) you were taught using digital resources. (Probe the specific content area in Integrated science, Mathematics, Pre-technical, and Agriculture/ Physics, Biology or Chemistry)","3. How often are digital resources used during your Science or mathematics lessons?(To probe how many lessons, digital resources have been used, in this term- Science, mathematics, Pre-technical, and Agriculture, Physics, Biology or Chemistry lessons)",4. Does your teacher allow you to interact with digital resources? (Probe on the mode of interaction with digital resources),"5.a. How do you feel when your teacher uses digital resources in the lesson? (Probe for interest, or easy understanding)","5.b. Can you share an example of a digital resource that made the lesson exciting, enjoyable, or easier to understand? (Probe for Integrated science, mathematics, pre-technical, and Agriculture, Physics, Biology or Chemistry)","6. Does your teacher give you assignments on the main idea learnt when using digital resources in a lesson? (If YES, explain how the assignment help you to recall the learnt idea)"
0,Ngererit Secondary School,Bomet,Konoin,Secondary School,Public,Day,Mixed Boys/Girls,Regular,Computer\nTelevision\nMobile Phones\nLaptops\n...,Chemistry - Learning about Metals\nMathematics...,Daily - for Mathematics using Scientific Calcu...,No. Learners only observe the outputs,Better and easy understanding of the concept b...,Showing a video of how the heart operates and ...,Yes. Using a phone at home to find out how to ...
1,Tenwek secondary,Bomet,Bomet central,Secondary School,Public,Boarding,Boys,Regular,"Computers, laptops, tablets, projectors, print...","Gas law using a projector, classification in b...",Digital resources used during double lessons i...,NaN,"Lesson is exciting. More understood, more inte...","Projection of types of electric bells, classif...","Yes. In physics to draw diagrams projected, to..."
2,Kamungei secondary,Bomet,Sotik,Secondary School,Public,Boarding,Mixed Boys/Girls,Regular,"Tablets, computers, projectors, laptops, phones","Projection of notes, videos of earthquakes in ...",Once in a while,"Yes, doing further research in the computer lab",Better understanding and tension of content. I...,In English teacher uses tv to show videos of s...,"Yes, search for more information in the comput..."
3,Kaplobotwa,Bomet,Bomet east,Secondary School,Public,Mixed Day/Boarding,Mixed Boys/Girls,Regular,"Computers, laptops,smartphones, tablets,calcul...","Matrices using computer,calculation of moralit...",During double lessons. Depends on if the topic...,Yes. With calculators and computers and teache...,"Enjoyable, understand better,","Computer, laptop, phone","Follow up questions, holiday assignments put i..."
4,Itembe,Bomet,Chepalungu,Secondary School,Public,Day,Mixed Boys/Girls,Regular,"Computers, laptops, phones, televisions, tablets","Waves in physics, morality in Chemistry, hormo...",Once in a month,Interact with computers in the lab to do assig...,"Helps understand more, makes one active, inter...","Projector, laptop, computer",Yes. Use computers to do assignments and searc...


In [169]:
ss1.tail()

,1.a. Name of School,1.b. County,1.c. Sub-County,2. Level of the school:,3. Category:,4.a. Type of school:,4.b. Type of school:,5. Nature of School:,"1. Name some of the digital resources your teachers use to teach integrated science, Mathematics, Pre-technical, and Agriculture, Physics, Biology or Chemistry lessons.","2. Tell me some specific lesson(s) you were taught using digital resources. (Probe the specific content area in Integrated science, Mathematics, Pre-technical, and Agriculture/ Physics, Biology or Chemistry)","3. How often are digital resources used during your Science or mathematics lessons?(To probe how many lessons, digital resources have been used, in this term- Science, mathematics, Pre-technical, and Agriculture, Physics, Biology or Chemistry lessons)",4. Does your teacher allow you to interact with digital resources? (Probe on the mode of interaction with digital resources),"5.a. How do you feel when your teacher uses digital resources in the lesson? (Probe for interest, or easy understanding)","5.b. Can you share an example of a digital resource that made the lesson exciting, enjoyable, or easier to understand? (Probe for Integrated science, mathematics, pre-technical, and Agriculture, Physics, Biology or Chemistry)","6. Does your teacher give you assignments on the main idea learnt when using digital resources in a lesson? (If YES, explain how the assignment help you to recall the learnt idea)"
95,Neboi Boys Sec.,Mandera,Khalalio,Secondary School,Public,Boarding,Boys,Regular,"Computer, projector, mobile phones, laptops",Agriculture -Birth process in a cow; Physics -...,Mathematics -Twice in a week; Biology -Twice a...,No,We feel happy and appreciate teacher's effort;...,"Laptop, projector, YouTube","Yes, we can recall the content learnt easily b..."
96,Dida Sec,Kilifi,Ganze,Secondary School,Public,Day,Mixed Boys/Girls,Regular,None,Never taught,Never used,Not used,Has never been used in class,None,None
97,m andagala,Vihiga,hamisi,Secondary School,Public,Mixed Day/Boarding,Mixed Boys/Girls,Regular,projector \nsmart phone \ntelevision,biology\ncomputer,need basis,yes,enjoy \ninteresting,project work\ncell,no
98,chaduma sec,Vihiga,sabatia,Secondary School,Public,Day,Mixed Boys/Girls,Regular,projector\nprinter\nlaptop,english during literature,per term,no,very interesting,english,no
99,st bakita siratsi sec,Vihiga,emuhaya,Secondary School,Public,NaN,Girls,Regular,biology,biology,it only when need arises,no,interesting,projector,no


In [170]:
#shape
shape=ss1.shape
print(f'Dataframe has {shape[0]} rows and {shape[1]} columns')

Dataframe has 100 rows and 15 columns


In [171]:
# columns
ss1.columns

Index(['1.a. Name of School', '1.b. County', '1.c. Sub-County',
       '2.  Level of the school:', '3. Category:', '4.a. Type of school: ',
       '4.b.  Type of school:', '5. Nature of School:',
       '1.  Name some of the digital resources your teachers use to teach integrated science, Mathematics, Pre-technical, and Agriculture, Physics, Biology or Chemistry lessons.',
       '2. Tell me some specific lesson(s) you were taught using digital resources. (Probe  the specific content area in Integrated science, Mathematics, Pre-technical, and Agriculture/ Physics, Biology or Chemistry) ',
       '3. How often are digital resources used during your Science or mathematics lessons?(To probe how many lessons, digital resources have been used, in this term-  Science, mathematics, Pre-technical, and Agriculture, Physics, Biology or Chemistry lessons) ',
       '4. Does your teacher allow you to interact with digital resources? (Probe on the mode of interaction with digital resources)  ',
   

In [172]:
#null values
ss1.isna().sum()

,0
1.a. Name of School,0
1.b. County,0
1.c. Sub-County,2
2. Level of the school:,0
3. Category:,0
4.a. Type of school:,1
4.b. Type of school:,1
5. Nature of School:,0
"1. Name some of the digital resources your teachers use to teach integrated science, Mathematics, Pre-technical, and Agriculture, Physics, Biology or Chemistry lessons.",4
"2. Tell me some specific lesson(s) you were taught using digital resources. (Probe the specific content area in Integrated science, Mathematics, Pre-technical, and Agriculture/ Physics, Biology or Chemistry)",4


## DATA CLEANING

1. COLUMN CONSISTENCY

In [24]:
js1=pd.read_excel('FGD_JS_cleaned.xlsx')
js2=pd.read_excel('HoI_JS_cleaned.xlsx')
js3=pd.read_excel('LO_JS_cleaned.xlsx')
questions=pd.read_excel('questionnaire_cleaned.xlsx')

In [6]:
js1.head()

,1.a. Name of School,1.b. County,1.c. Sub-County,2. Level of the school:,3. Category:,4.a. Type of school:,4.b. Type of school:,5. Nature of School:,"1. Name some of the digital resources your teachers use to teach integrated science, Mathematics, Pre-technical, and Agriculture, Physics, Biology or Chemistry lessons.","2. Tell me some specific lesson(s) you were taught using digital resources. (Probe the specific content area in Integrated science, Mathematics, Pre-technical, and Agriculture/ Physics, Biology or Chemistry)","3. How often are digital resources used during your Science or mathematics lessons?(To probe how many lessons, digital resources have been used, in this term- Science, mathematics, Pre-technical, and Agriculture, Physics, Biology or Chemistry lessons)",4. Does your teacher allow you to interact with digital resources? (Probe on the mode of interaction with digital resources),"5.a. How do you feel when your teacher uses digital resources in the lesson? (Probe for interest, or easy understanding)","5.b. Can you share an example of a digital resource that made the lesson exciting, enjoyable, or easier to understand? (Probe for Integrated science, mathematics, pre-technical, and Agriculture, Physics, Biology or Chemistry)","6. Does your teacher give you assignments on the main idea learnt when using digital resources in a lesson? (If YES, explain how the assignment help you to recall the learnt idea)"
0,Ngererit Primary and Junior School,Bomet,Konoin,Junior School,Public,Day,Mixed Boys/Girls,Regular,Phones\nLaptops\nComputers\nScientific Calcula...,Laptop - Teaching Plants in Integrated Science...,About 2 to 3 lessons per week in lessons of In...,Rarely. Learners shown the outcome on the phones,"Feeling happy, better understanding of the con...",Easier to understanding by seeing photos of or...,Rarely
1,St. Mary Mixed Day and Boarding Primary & Juni...,Bomet,Chepalungu,Junior School,Private,Mixed Day/Boarding,Mixed Boys/Girls,Regular,a) Laptops\nb) Computers\nc) Mobile phones\nd)...,"Very often, at least once a week\na) Organic F...",Yes. At least once a week,Yes. To search for content in the internet,"Excited, happy, learning interesting hence mor...","a) Types of Computers, Parts of a Computer - P...",Yes. Learners do further research
2,Kiplabotwa Primary & Junior School,Bomet,Bomet East,Junior School,Public,Day,Mixed Boys/Girls,Regular & SNE,a) Tablets\nb) Laptops\nc) Note Books\nd) Desk...,"a) Structure of an Atom, Living Things - Integ...",Twice per week majorly in Mathematics and Scie...,Yes. Through google searches and YouTube,Lessons interesting. More and better understan...,a) Living Things - Integrated Science\nb) Stru...,Yes.
3,Sotik JS,Bomet,Sotik,Junior School,Public,Day,Mixed Boys/Girls,Regular,"Computers, tablets, markbook, smartphones, lap...","Lab safety in integrated science , how to crea...",Regularly,Yes. We are given tablets to search for digita...,"Happy, enjoyable, interesting and makes it eas...",Tablets and use of projector to project video,Given a link to copy and use it later to acces...
4,Chelsa Comprehensive JS Academy,Bomet,Bomet central,Junior School,Private,Mixed Day/Boarding,Mixed Boys/Girls,Regular,"Desktops, laptops, projectors, tablets, markbo...",Scratch and composite materials in pre-technic...,Average of 4 times per week. But for mathemati...,Interact with computer in computer lab to disc...,"Motivated, lesson is lively, understand more,f...","Computers, smartphones, laptops, projectors",Yes. Assignment to search for more information...


In [7]:
js1.columns

Index(['1.a. Name of School', '1.b. County', '1.c. Sub-County',
       '2.  Level of the school:', '3. Category:', '4.a. Type of school: ',
       '4.b.  Type of school:', '5. Nature of School:',
       '1.  Name some of the digital resources your teachers use to teach integrated science, Mathematics, Pre-technical, and Agriculture, Physics, Biology or Chemistry lessons.',
       '2. Tell me some specific lesson(s) you were taught using digital resources. (Probe  the specific content area in Integrated science, Mathematics, Pre-technical, and Agriculture/ Physics, Biology or Chemistry) ',
       '3. How often are digital resources used during your Science or mathematics lessons?(To probe how many lessons, digital resources have been used, in this term-  Science, mathematics, Pre-technical, and Agriculture, Physics, Biology or Chemistry lessons) ',
       '4. Does your teacher allow you to interact with digital resources? (Probe on the mode of interaction with digital resources)  ',
   

In [8]:
# ensuring column consistency
rename_dict = {
    '1.  Name some of the digital resources your teachers use to teach integrated science, Mathematics, Pre-technical, and Agriculture, Physics, Biology or Chemistry lessons.': 'Digital resources used by teachers',
    '2. Tell me some specific lesson(s) you were taught using digital resources. (Probe  the specific content area in Integrated science, Mathematics, Pre-technical, and Agriculture/ Physics, Biology or Chemistry) ': 'Lessons taught using digital resources',
    '3. How often are digital resources used during your Science or mathematics lessons?(To probe how many lessons, digital resources have been used, in this term-  Science, mathematics, Pre-technical, and Agriculture, Physics, Biology or Chemistry lessons) ': 'Frequency of digital resource use in lessons',
    '4. Does your teacher allow you to interact with digital resources? (Probe on the mode of interaction with digital resources)  ': 'Student interaction with digital resources',
    '5.a. How do you feel when your teacher uses digital resources in the lesson? (Probe for interest, or easy understanding) ': 'Student feelings when teacher uses digital resources',
    '5.b.  Can you share an example of a digital resource that made the lesson exciting, enjoyable, or easier to understand? (Probe for Integrated science, mathematics,  pre-technical, and Agriculture, Physics, Biology or Chemistry)': 'Example of effective digital resource in lesson',
    '6. Does your teacher give you assignments on the main idea learnt when using digital resources in a lesson? (If YES, explain how the assignment help you to recall the learnt idea)': 'Assignments based on digital resource lessons'
}

js1 = js1.rename(columns=rename_dict)

In [9]:
#checking
js1.columns

Index(['1.a. Name of School', '1.b. County', '1.c. Sub-County',
       '2.  Level of the school:', '3. Category:', '4.a. Type of school: ',
       '4.b.  Type of school:', '5. Nature of School:',
       'Digital resources used by teachers',
       'Lessons taught using digital resources',
       'Frequency of digital resource use in lessons',
       'Student interaction with digital resources',
       'Student feelings when teacher uses digital resources',
       'Example of effective digital resource in lesson',
       'Assignments based on digital resource lessons'],
      dtype='object')

In [10]:
#checking datatypes
js1.dtypes

,0
1.a. Name of School,object
1.b. County,object
1.c. Sub-County,object
2. Level of the school:,object
3. Category:,object
4.a. Type of school:,object
4.b. Type of school:,object
5. Nature of School:,object
Digital resources used by teachers,object
Lessons taught using digital resources,object


In [11]:
js1.head()

,1.a. Name of School,1.b. County,1.c. Sub-County,2. Level of the school:,3. Category:,4.a. Type of school:,4.b. Type of school:,5. Nature of School:,Digital resources used by teachers,Lessons taught using digital resources,Frequency of digital resource use in lessons,Student interaction with digital resources,Student feelings when teacher uses digital resources,Example of effective digital resource in lesson,Assignments based on digital resource lessons
0,Ngererit Primary and Junior School,Bomet,Konoin,Junior School,Public,Day,Mixed Boys/Girls,Regular,Phones\nLaptops\nComputers\nScientific Calcula...,Laptop - Teaching Plants in Integrated Science...,About 2 to 3 lessons per week in lessons of In...,Rarely. Learners shown the outcome on the phones,"Feeling happy, better understanding of the con...",Easier to understanding by seeing photos of or...,Rarely
1,St. Mary Mixed Day and Boarding Primary & Juni...,Bomet,Chepalungu,Junior School,Private,Mixed Day/Boarding,Mixed Boys/Girls,Regular,a) Laptops\nb) Computers\nc) Mobile phones\nd)...,"Very often, at least once a week\na) Organic F...",Yes. At least once a week,Yes. To search for content in the internet,"Excited, happy, learning interesting hence mor...","a) Types of Computers, Parts of a Computer - P...",Yes. Learners do further research
2,Kiplabotwa Primary & Junior School,Bomet,Bomet East,Junior School,Public,Day,Mixed Boys/Girls,Regular & SNE,a) Tablets\nb) Laptops\nc) Note Books\nd) Desk...,"a) Structure of an Atom, Living Things - Integ...",Twice per week majorly in Mathematics and Scie...,Yes. Through google searches and YouTube,Lessons interesting. More and better understan...,a) Living Things - Integrated Science\nb) Stru...,Yes.
3,Sotik JS,Bomet,Sotik,Junior School,Public,Day,Mixed Boys/Girls,Regular,"Computers, tablets, markbook, smartphones, lap...","Lab safety in integrated science , how to crea...",Regularly,Yes. We are given tablets to search for digita...,"Happy, enjoyable, interesting and makes it eas...",Tablets and use of projector to project video,Given a link to copy and use it later to acces...
4,Chelsa Comprehensive JS Academy,Bomet,Bomet central,Junior School,Private,Mixed Day/Boarding,Mixed Boys/Girls,Regular,"Desktops, laptops, projectors, tablets, markbo...",Scratch and composite materials in pre-technic...,Average of 4 times per week. But for mathemati...,Interact with computer in computer lab to disc...,"Motivated, lesson is lively, understand more,f...","Computers, smartphones, laptops, projectors",Yes. Assignment to search for more information...


- JS2

In [12]:
js2.head()

,1.a. Name of the school:,JS/Sec,1.b. Name of your County,1.c. Sub-County,2. Head of Institution’s gender:,3.a. Type of school:,3.b. Type of school:,4.a. Category of School:,4.b. Nature of School:,5. For how long have you been a Head of Institution?,...,4.b. Tablet,5.a. Printers (Quantity),5.b. Printers,6.a. Digital cameras (Quantity),6.b. Digital cameras,7.a. School mobile phone(s) (Quantity),7.b. School mobile phone(s),8.a. Software applications (phet) (Quantity),8.b. Software applications,10. Internet connectivity
0,Sotik Primary & Junior School,JS,Bomet,Sotik,Male,Day,Mixed Boys/Girls,Public,Regular,6-10 years,...,Not Adequate,2,Not Adequate,0,Not adequate,0,Not Adequate,0,Not Adequate,Available
1,Kiplabotwa Primary & Junior School,JS,Bomet,Bomet East,Male,Day,Mixed Boys/Girls,Public,Regular & SNE,6-10 years,...,Not Adequate,1,Not Adequate,0,Not adequate,0,Not Adequate,0,Not Adequate,Available
2,St. Mary's Mixed Day and Boarding Primary & Ju...,JS,Bomet,Chepalungu,Female,Mixed Day/Boarding,Mixed Boys/Girls,Private,Regular,6-10 years,...,Not Adequate,5,Not Adequate,1,Not adequate,0,Not Adequate,0,Not Adequate,Available
3,Chelsa Academy - Primary & Junior School,JS,Bomet,Bomet Central,Male,Mixed Day/Boarding,Mixed Boys/Girls,Private,Regular,1-5 years,...,Not Adequate,2,Adequate,0,Not adequate,0,Not Adequate,0,Not Adequate,Available
4,Victory Junior Academy,JS,Busia,Teso South,Male,Mixed Day/Boarding,Mixed Boys/Girls,Private,Regular,1-5 years,...,Not Adequate,2,Not Adequate,1,Not adequate,1,Adequate,2,Not Adequate,Available


In [13]:
js2.columns

Index(['1.a. Name of the school: ', 'JS/Sec', '1.b. Name of your County',
       '1.c. Sub-County', '2. Head of Institution’s gender:',
       '3.a. Type of school:', '3.b. Type of school:',
       '4.a. Category of School:', '4.b. Nature of School:',
       '5. For how long have you been a Head of Institution?',
       '6.a. Do your STEM teachers participate in professional development  training related to their subjects?',
       '6.b. Probe on how often  this happens and the kind of training',
       '7.a. Do you feel the training opportunities adequately equip teachers with the skills to use digital Resources in STEM effectively?',
       '7.b. Probe for why or why not',
       '8. What measures do you have in place to ensure teachers are updated with the current digital resources for STEM?  ( Probe about AI, virtual labs among others)',
       '9.a. Do your STEM teachers incorporate digital resources in their lessons?',
       '9.b. (Probe on frequency and cite instances) (In your

In [14]:
#renaming columns
rename_dict = {
    '5. For how long have you been a Head of Institution?': 'Tenure as Head of Institution',
    '6.a. Do your STEM teachers participate in professional development  training related to their subjects?': 'STEM teacher participation in professional development',
    '6.b. Probe on how often  this happens and the kind of training': 'Frequency and type of STEM teacher training',
    '7.a. Do you feel the training opportunities adequately equip teachers with the skills to use digital Resources in STEM effectively?': 'Adequacy of training for digital resource use in STEM',
    '7.b. Probe for why or why not': 'Reasoning for adequacy or inadequacy of training',
    '8. What measures do you have in place to ensure teachers are updated with the current digital resources for STEM?  ( Probe about AI, virtual labs among others)': 'Measures to keep teachers updated on digital resources',
    '9.a. Do your STEM teachers incorporate digital resources in their lessons?': 'STEM teacher incorporation of digital resources',
    '9.b. (Probe on frequency and cite instances) (In your opinion, how confident are your teachers in using technology and digital resources in their teaching)': 'Frequency and confidence in technology use by teachers',
    '10. Give examples of digital resources your teachers commonly use in STEM classes? (Probe on both hardware (laptop, projectors) and software (phet))': 'Examples of digital resources used in STEM classes',
    '11.a. Do you evaluate the use of digital resources in the classroom?': 'Evaluation of digital resource use in classroom',
    '11.b. Probe on how': 'Methods of evaluating digital resource use',
    '12. What challenges do your teachers face when integrating digital resources into their STEM lessons?(Probe on any areas that teachers need support)': 'Challenges faced by teachers integrating digital resources',
    '13. How are you addressing the challenges identified above in order to enhance the teacher efficacy? (probe on  technical assistance, maintenance and replacement of digital equipment funding for resources)': 'Measures to address challenges in digital resource integration',
    '14.What is your schools’ policy on use of digital resources in teaching and learning?(Probes on any guidelines/vision on the use of digital resources (phones))': 'School policy on digital resource use'
}

# To rename in your DataFrame, for example if it is called `heads`:
js2 = js2.rename(columns=rename_dict)

In [15]:
#checking
js2.columns

Index(['1.a. Name of the school: ', 'JS/Sec', '1.b. Name of your County',
       '1.c. Sub-County', '2. Head of Institution’s gender:',
       '3.a. Type of school:', '3.b. Type of school:',
       '4.a. Category of School:', '4.b. Nature of School:',
       'Tenure as Head of Institution',
       'STEM teacher participation in professional development',
       'Frequency and type of STEM teacher training',
       'Adequacy of training for digital resource use in STEM',
       'Reasoning for adequacy or inadequacy of training',
       'Measures to keep teachers updated on digital resources',
       'STEM teacher incorporation of digital resources',
       'Frequency and confidence in technology use by teachers',
       'Examples of digital resources used in STEM classes',
       'Evaluation of digital resource use in classroom',
       'Methods of evaluating digital resource use',
       'Challenges faced by teachers integrating digital resources',
       'Measures to address challenge

- THIRD DATAFRAME

In [16]:
js3.head()

,1.a. School,1.b. County,1.c. Sub-County,2.a. Date,2.b. Time : From,2.c. Time : To,3.a. Name of the teacher,3.b. TSC No:,4. Gender,5. Professional qualifications:,...,"Innovative Use of Digital Resource [5. Digital resources were well-aligned with learning outcomes and learner’s needs (e.g., age-appropriate content, differentiated instruction).]",Assessment [6. The teacher provided timely and constructive feedback based on the learners interaction with the digital resource],Assessment [7. Learners had the opportunity to give feedback on the concept learnt based on the their interaction with the digital resource],Classroom Management [8. Teacher managed transitions in activities involving digital resources seamlessly],"Classroom Management [9. Learners remained focused on the task, as the teacher guides or redirects their behavior]","Digital Fluency [10. Teacher demonstrated proficiency in using digital resources to achieve lesson learning outcomes/objectives (e.g., confident navigation of software).]","Digital Fluency [11. Teacher resolved technical issues effectively during the lesson (e.g., troubleshooting connectivity problems-If none rate 3)]",1. How prepared was the teacher in terms of accessing and utilizing digital resources?,"2. Were there any notable challenges in using digital resources? If yes, describe them.",3. Suggest area(s) in which the teacher should be supported to enhance his/her skills in the use of digital resources in the learning process
0,Busulere js,Busia,samia,2025-02-12,08:40:00,09:20:00,Susan Ogomaa,928554,Female,B Ed,...,3,3,3,3,3,2,3,had the digital resources set ready for the le...,surface for projection of the digital resource...,enhance the pedagogical skills of the teacher
1,EMBU URBAN JSS,Embu,EMBU WEST,2025-02-10,10:40:00,11:20:00,Eric Kinyua,1020167,Male,B Ed,...,3,2,3,1,2,2,2,Tablets use available but no prior configurati...,Yes. Some learners did not know the WIFI passw...,i. Test devices before the lesson. ii. Ensure ...
2,Bonanza School,Embu,Embu West,2025-02-13,11:40:00,12:20:00,Dennis Munene,908111,Male,Diploma in Ed,...,2,2,1,1,2,1,1,The teacher was being assisted by an ICT techn...,i. Weak internet connectivity;\nii. Learners w...,The teacher should enhance his skills on ICT t...
3,MPIS,Homa Bay,MBITA,2025-02-11,09:37:00,10:18:00,Paul Nyangacha,921879,Male,B Ed,...,2,2,3,2,3,3,3,Fair prepared,No,Preparation of ICT lesson where learners have ...
4,Osodo JS,Homa Bay,Rachuonyo Noth,2025-02-13,12:03:00,12:34:00,Shem Onyango,684304,Male,Diploma in Ed,...,2,1,1,2,2,2,2,Limited preparation,No,Training on effective Interactive learning exp...


In [19]:
js3.columns

Index(['1.a. School', '1.b. County', '1.c. Sub-County', '2.a. Date',
       '2.b. Time : From', '2.c. Time : To', '3.a. Name of the teacher ',
       '3.b. TSC No:  ', '4. Gender', '5. Professional qualifications:  ',
       '6.1. Teaching subjects trained in (College subjects): ',
       '6.2. Teaching subjects trained in (College subjects):  ',
       '7. Employer ', '8. Have you attended any teacher training in STEM? ',
       '9. If yes, how many times have you undergone the training? ',
       '10. Grade/Form', 'JS/SS', '11. Subject ', '12. Strand/Topic ',
       '13. Sub-strand/Sub-Topic ', '14. Number of Learners: Total ',
       '15. Lesson Learning outcomes ', 'Learning Resources ',
       '16. Key Inquiry Question (KIQ) ',
       'Statements  [1. The teacher had a scheme of work]',
       'Statements  [2. Did the column of resources in the Schemes of work indicate digital resources]',
       'Statements  [3. The teacher had a lesson plan prepared as per the scheme of work]',


In [20]:
#standardizing columns
rename_dict = {
    '6.1. Teaching subjects trained in (College subjects): ': 'Teaching subjects trained (1)',
    '6.2. Teaching subjects trained in (College subjects):  ': 'Teaching subjects trained (2)',
    '7. Employer ': 'Employer',
    '8. Have you attended any teacher training in STEM? ': 'Attended STEM teacher training',
    '9. If yes, how many times have you undergone the training? ': 'Number of STEM trainings attended',
    '10. Grade/Form': 'Grade or Form',
    'JS/SS': 'School Level (JS/SS)',
    '11. Subject ': 'Subject',
    '12. Strand/Topic ': 'Strand or Topic',
    '13. Sub-strand/Sub-Topic ': 'Sub-strand or Sub-Topic',
    '14. Number of Learners: Total ': 'Total learners',
    '15. Lesson Learning outcomes ': 'Lesson learning outcomes',
    'Learning Resources ': 'Learning resources',
    '16. Key Inquiry Question (KIQ) ': 'Key Inquiry Question (KIQ)',
    'Statements  [1. The teacher had a scheme of work]': 'Teacher had scheme of work',
    'Statements  [2. Did the column of resources in the Schemes of work indicate digital resources]': 'Schemes of work indicated digital resources',
    'Statements  [3. The teacher had a lesson plan prepared as per the scheme of work]': 'Lesson plan prepared as per scheme of work',
    'Statements  [4. Did the Lesson plan include digital resources?]': 'Lesson plan included digital resources',
    'Statements  [5. Are the digital resources aligned to the lesson learning outcomes/objectives?]': 'Digital resources aligned to learning outcomes',
    'Learner Engagement  [1. The teacher used digital resources to promote active participation by learners (group task, interactive tools)]': 'Teacher used digital resources for active participation',
    'Learner Engagement  [2. Learners interacted with digital resources effectively to enhance understanding of STEM concepts (e.g., simulations, virtual labs).]': 'Learners interacted with digital resources effectively',
    'Innovative Use of Digital Resource [3. Teacher’s creativity of integrating digital resource in the lesson was evident(e.g., using locally relevant examples, customizing software settings, contextualizing with learners’ experiences)]': 'Teacher creativity in integrating digital resources',
    'Innovative Use of Digital Resource [4. The learners were able to relate the digital resource to their experiences. (e.g., using locally relevant examples, customizing software settings)]': 'Learners related digital resources to experiences',
    'Innovative Use of Digital Resource [5. Digital resources were well-aligned with learning outcomes and learner’s needs (e.g., age-appropriate content, differentiated instruction).]': 'Digital resources aligned with needs and outcomes',
    'Assessment  [6. The teacher provided timely and constructive feedback based on the learners interaction with the digital resource]': 'Teacher provided feedback on digital resource use',
    'Assessment  [7. Learners had the opportunity to give feedback on the concept learnt based on the their interaction with the digital resource]': 'Learners provided feedback on digital resource learning',
    'Classroom Management  [8. Teacher managed transitions in activities involving  digital resources seamlessly]': 'Teacher managed digital resource activities',
    'Classroom Management  [9. Learners remained focused on the task, as the teacher guides or redirects their behavior]': 'Learner focus during digital resource activities',
    'Digital Fluency [10. Teacher demonstrated proficiency in using digital resources to achieve lesson learning outcomes/objectives (e.g., confident navigation of software).]': 'Teacher proficiency with digital resources',
    'Digital Fluency [11. Teacher resolved technical issues effectively during the lesson (e.g., troubleshooting connectivity problems-If none rate 3)]': 'Teacher resolved technical issues with digital resources',
    '1. How prepared was the teacher in terms of accessing and utilizing digital resources? ': 'Teacher preparedness for digital resource use',
    '2. Were there any notable challenges in using digital resources? If yes, describe them. ': 'Challenges in using digital resources',
    '3.  Suggest area(s) in which the teacher should be supported to enhance his/her skills in the use of digital resources in the learning process': 'Suggested support areas for digital resource use'
}

# Suppose your DataFrame is called `observations`:
js3 = js3.rename(columns=rename_dict)

In [21]:
#checking
js3.columns

Index(['1.a. School', '1.b. County', '1.c. Sub-County', '2.a. Date',
       '2.b. Time : From', '2.c. Time : To', '3.a. Name of the teacher ',
       '3.b. TSC No:  ', '4. Gender', '5. Professional qualifications:  ',
       'Teaching subjects trained (1)', 'Teaching subjects trained (2)',
       'Employer', 'Attended STEM teacher training',
       'Number of STEM trainings attended', 'Grade or Form',
       'School Level (JS/SS)', 'Subject', 'Strand or Topic',
       'Sub-strand or Sub-Topic', 'Total learners', 'Lesson learning outcomes',
       'Learning resources', 'Key Inquiry Question (KIQ)',
       'Teacher had scheme of work',
       'Schemes of work indicated digital resources',
       'Lesson plan prepared as per scheme of work',
       'Lesson plan included digital resources',
       'Digital resources aligned to learning outcomes',
       'Teacher used digital resources for active participation',
       'Learners interacted with digital resources effectively',
       'Teache

- FOURTH DATAFRAME

In [28]:
questions.columns

Index(['1. Have you been observed in class (for this  research) ',
       '2.a. Name of School ', '2.b. County', '2.c. Sub-County', '3. Gender',
       '4.a. Type of school: Category of school ',
       '4.b. Type of school: Nature of school ', '4.c. Type of school:',
       '5. Indicate your age range: ', '6. Teaching experience? ',
       '7. Highest level of academic qualification: ',
       '8. How many lessons do you teach per week? ', '9. Employer:',
       '10. Have you undergone any teacher professional development training? ',
       'If YES, for how many times? ',
       'Statement  [1. I source for digital resources to use during my lesson]',
       'Statement  [2. I collaborate with colleagues in sourcing for digital resources]',
       'Statement  [3. I develop digital resources for use during the lessons]',
       'Statement  [4. I try-out the digital resources before the lessons]',
       'Statement  [5. The school provides adequate digital tools for use in the lessons]'

In [29]:
#standardizing column names
rename_dict = {
    '7. Highest level of academic qualification: ': '7. Academic qualification (highest)',
    '8. How many lessons do you teach per week? ': '8. Lessons per week',
    '9. Employer:': '9. Employer',
    '10. Have you undergone any teacher professional development training? ': '10. Attended professional development training',
    'If YES, for how many times? ': 'Teacher training frequency',
    'Statement  [1. I source for digital resources to use during my lesson]': 'Statement 1. Sources digital resources',
    'Statement  [2. I collaborate with colleagues in sourcing for digital resources]': 'Statement 2. Collaborates in sourcing digital resources',
    'Statement  [3. I develop digital resources for use during the lessons]': 'Statement 3. Develops digital resources',
    'Statement  [4. I try-out the digital resources before the lessons]': 'Statement 4. Tries out digital resources before lessons',
    'Statement  [5. The school provides adequate digital tools for use in the lessons]': 'Statement 5. School provides adequate digital tools',
    'Statement  [6. The school administration supports me to use digital resources for learning]': 'Statement 6. School administration supports digital resources',
    'Statement  [7. The professional development opportunities provided enhance my knowledge and skills in using digital resources]': 'Statement 7. Professional development enhances digital resource use',
    'Statement [8. I give all learners equal opportunities in the use of the digital resources]': 'Statement 8. Equal learner opportunities with digital resources',
    'Statement [9. I ask learners questions to guide learners’ interaction with the digital resources]': 'Statement 9. Guides learners’ interaction with questions',
    'Statement [10. I engage my learners in group discussions to share their ideas with their peers as they interact with the digital resources]': 'Statement 10. Group discussions with digital resources',
    'Statement [11. I encourage learners to ask questions on the concepts developed using the digital resources]': 'Statement 11. Encourages learner questions on digital resources',
    'Statement [12. I give learners enough time to interact with digital resources]': 'Statement 12. Time for learner interaction with digital resources',
    'Statement [13. I give learners assignments on concepts learned using  the digital resources]': 'Statement 13. Assignments on digital resources concepts',
    'Statement [14. I use digital resources   to assess learning]': 'Statement 14. Uses digital resources to assess learning',
    '15. Which digital resources do you commonly use in your lessons? \na) Hardware:': '15a. Commonly used digital resources (Hardware)',
    '15. Which digital resources do you commonly use in your lessons? \nb)  Software (Apps/videos/animation/simulations/virtual labs/Audio etc.):': '15b. Commonly used digital resources (Software)',
    '16.  How has the use of digital resources influenced my teaching practices?  ': '16. Influence of digital resources on teaching',
    '17. What type of support do you receive from the school leadership in the use of digital resources? ': '17. School leadership support for digital resource use',
    '18. (a)  Briefly explain how teacher  professional development has helped you in the use digital resources ': '18a. Impact of professional development on digital resource use',
    '18. (b)  Suggest training needs to be addressed on the use of digital resources in future Teacher Professional Development program ': '18b. Suggested future training needs for digital resource use'
}

# Usage example (assuming your DataFrame is called `teachers`)
questions = questions.rename(columns=rename_dict)

In [30]:
#checking
questions.columns

Index(['1. Have you been observed in class (for this  research) ',
       '2.a. Name of School ', '2.b. County', '2.c. Sub-County', '3. Gender',
       '4.a. Type of school: Category of school ',
       '4.b. Type of school: Nature of school ', '4.c. Type of school:',
       '5. Indicate your age range: ', '6. Teaching experience? ',
       '7. Academic qualification (highest)', '8. Lessons per week',
       '9. Employer', '10. Attended professional development training',
       'Teacher training frequency', 'Statement 1. Sources digital resources',
       'Statement 2. Collaborates in sourcing digital resources',
       'Statement 3. Develops digital resources',
       'Statement 4. Tries out digital resources before lessons',
       'Statement 5. School provides adequate digital tools',
       'Statement 6. School administration supports digital resources',
       'Statement 7. Professional development enhances digital resource use',
       'Statement 8. Equal learner opportunities w